# Load

## Libs

In [1]:
suppressPackageStartupMessages({
    source('../libs.R')
    source('../utils.R')    
    source('../Figures/colors.R')    
    source('../utils_mapping.R')
    source('../utils_plotting.R')
})


## data

In [2]:
ddres_costim <- readRDS('/data/srlab2/ik936/Roche/data/cache/stim_revisions_ddres_v1.rds')

In [3]:
cluster_names_df <- unique(dplyr::select(readRDS('/data/srlab2/ik936/Roche/data/cache/obj_fibroblasts/meta_data.rds'), Cluster, Cluster_name))

In [ ]:
## Get pseudobulk expression to plot in heatmap 
obj_costim <- readRDS('/data/srlab2/ik936/Roche/data/cache/revision1_stim_object.rds')
pb_costim <- presto::collapse_counts(obj_costim$exprs_raw, obj_costim$meta_data, c('DonorID', 'Condition', 'Tissue'))

## Z-score within tissue 
pb_costim$exprs_scaled <- pb_costim$meta_data %>% split(.$Tissue) %>% map(function(.SD) {
    scaleData(pb_costim$exprs_norm[, rownames(.SD)], margin = 1, thresh = 5)
}) %>% 
    purrr::reduce(Matrix::cbind2)
pb_costim$exprs_scaled <- pb_costim$exprs_scaled[, colnames(pb_costim$exprs_norm)]
pb_costim$exprs_scaled[which(is.na(pb_costim$exprs_scaled))] <- 0

In [ ]:
obj_costim_fls <- readRDS('/data/srlab2/ik936/Roche/data/cache/revision1_stim_object_synovium.rds')
obj_costim_lung <- readRDS('/data/srlab2/ik936/Roche/data/cache/revision1_stim_object_lung.rds')

# Cell counts

In [ ]:
fig.size(4, 6)
p_counts <- data.table(obj_costim$meta_data)[, .N, by = .(Condition, DonorID, Tissue)] %>% 
    ggplot(aes(DonorID, N, fill = Condition)) + 
        geom_bar(stat = 'identity', position = position_dodge()) + 
        facet_grid(Tissue~., scales = 'free') + 
        coord_flip() + 
        scale_y_log10() + 
        scale_fill_tableau() + 
        geom_hline(yintercept = c(1e3), linetype = 2) + 
        labs(y = 'Number cells') + 
        NULL

p_counts

# UMAPs

In [ ]:
fig.size(4, 10)
p_umap_fls <- do_scatter(obj_costim_fls$U$embedding, obj_costim_fls$meta_data, 'DonorID', quo(Condition), do_labels=FALSE, no_guides=FALSE, nrow=1) + 
    labs(title = 'Synovial fibroblasts') + 
    theme(
        legend.position="bottom",
        plot.title = element_text(size=12),
        axis.ticks = element_blank(), 
        axis.text = element_blank(),
#         axis.title.x = element_blank()
    ) + 
    NULL
p_umap_fls

In [ ]:
fig.size(4, 10)
p_umap_lung <- do_scatter(obj_costim_lung$U$embedding, obj_costim_lung$meta_data, 'DonorID', quo(Condition), do_labels=FALSE, no_guides=FALSE, palette_use = tableau_color_pal()(6)[4:6], nrow=1) + 
    labs(title = 'Lung fibroblasts') + 
    theme(
        legend.position="bottom",
        plot.title = element_text(size=12),
        axis.ticks = element_blank(), 
        axis.text = element_blank(),
#         axis.title.y = element_blank()
    ) + 
    NULL
p_umap_lung

# Condition-vs-condition

In [ ]:
# data.table(ddres_costim)[padj < .05 & stat > 0, .SD[order(-log2FoldChange)][1:10], by = .(Tissue, Condition)]
#[, .N, by = .(Tissue, Condition)]
# data.table(ddres_costim)[padj < .05 & stat > 0, .SD[order(-stat)][1:10], by = .(Tissue, Condition)]

# genes_plot_tcells <- data.table(ddres_costim)[padj < .01 & log2FoldChange > 2, .SD[order(-stat)], by = .(Tissue, Condition)][Condition == 'Tcells', unique(feature)]
# genes_plot_ecs <- data.table(ddres_costim)[padj < .01 & log2FoldChange > 2, .SD[order(-stat)], by = .(Tissue, Condition)][Condition == 'ECs', unique(feature)]


genes_plot <- data.table(ddres_costim)[padj < .01 & log2FoldChange > 2, .SD[order(-stat)], by = .(Condition)][, unique(feature)]
# genes_highlight <- data.table(ddres_costim)[padj < .01 & log2FoldChange > 2, .SD[order(-stat)][1:5], by = .(Tissue, Condition)][, unique(feature)] 
# genes_highlight <- ddres_costim %>% 
#     subset(padj < .01 & log2FoldChange > 2) %>% 
#     dplyr::select(Tissue, Condition, feature, log2FoldChange) %>% 
#     tidyr::spread(Tissue, log2FoldChange) %>% 
#     subset(!is.na(lung) & !is.na(fls)) %>% 
#     split(.$Condition) %>% 
#     map(function(.SD) {
#         .SD %>% 
#             arrange(-(fls*lung)) %>% 
#             head(10)
#     }) %>% 
#     bind_rows() %>% 
#     with(unique(feature)) 

genes_highlight <- c('CXCL10', 'CXCL11', 'CCL19', 'IGFBP2', 'ZBTB16', 'CCND2')

# genes_highlight <- union(genes_highlight, c('CXCL10', 'CCL19', 'SPARC'))
row_o <- pb_costim$meta_data %>% arrange(Condition, Tissue) %>% rownames()
mat <- t(pb_costim$exprs_scaled[genes_plot, row_o])
rownames(mat) <- glue('{Tissue} {Condition}', .envir = pb_costim$meta_data[row_o, ])


In [ ]:
anno_cols <- data.table(ddres_costim)[feature %in% genes_plot][, .(score = mean(log2FoldChange)), by = .(feature, Condition)][, .SD[order(-score)][1], by = feature][, .(feature, Condition)] %>% 
    data.frame() %>% 
    tibble::column_to_rownames('feature') %>% 
    arrange(Condition)
# anno_cols <- anno_cols[colnames(mat), , drop = FALSE]
mat <- mat[, rownames(anno_cols), drop = FALSE]

In [ ]:
fig.size(5, 8)
plt_df <- ddres_costim %>% 
#     dplyr::select(Tissue, Condition, SCORE=stat, feature) %>% 
    dplyr::select(Tissue, Condition, SCORE=log2FoldChange, feature) %>% 
    tidyr::spread(Condition, SCORE, fill = 0) %>% 
    dplyr::mutate(Tissue = c(fls='Synovium', lung='Lung')[Tissue])

plt_df_highlight <- anno_cols %>%  
    tibble::rownames_to_column('feature') %>% 
    subset(feature %in% genes_highlight) %>% 
    left_join(plt_df) %>% 
    subset(
        (Condition == 'ECs' & ECs > 1.5) | (Condition == 'Tcells' & Tcells > 1.5)
    )

p_scatter_condition <- ggplot(plt_df, aes(Tcells, ECs)) +
    facet_wrap(~Tissue, scales = 'free') + 
    geom_hline(yintercept = 0, linetype = 2) + 
    geom_vline(xintercept = 0, linetype = 2) + 
    geom_abline(linetype = 2) + 
    geom_point(shape = '.', position = position_jitter(height = .05, width = .05), alpha = .6, color = 'grey') + 
    geom_point(
        data = plt_df_highlight, 
        shape = 16, size = 1, color = muted('red')
    ) + 
    geom_text_repel(
        data = plt_df_highlight, 
        aes(label = feature),
        max.overlaps = 20,
        color = muted('red')
    ) + 
    labs(title = 'Compare Conditions') + 
    scale_color_manual(values = palette_global[c('Yes', 'No')]) + 
    guides(color = 'none') + 
    labs(y = 'Endothelial vs Control (logFC)', x = 'T cell supernatant vs Control (logFC)') + 
    NULL

p_scatter_condition

In [ ]:
genes_test <- data.table(ddres_costim)[pvalue < .01, unique(feature)]
ddres_costim %>% 
    subset(feature %in% genes_test) %>% 
#     dplyr::select(Tissue, Condition, SCORE=stat, feature) %>% 
    dplyr::select(Tissue, Condition, SCORE=log2FoldChange, feature) %>% 
    tidyr::spread(Condition, SCORE) %>% 
    split(.$Tissue) %>% 
    map(function(.SD) {
        broom::tidy(cor.test(~ ECs + Tcells, .SD, method = 'spearman'))
    }) %>% 
    bind_rows(.id = 'Condition')

# Focused corplot (T cells vs C11)

In [ ]:
plt_df <- readRDS('/data/srlab2/ik936/Roche/data/cache/obj_fibroblasts/effects_nested.rds') %>% subset(Tissue %in% c('Lung', 'Synovium')) %>% 
    inner_join(
        ddres_costim %>% dplyr::mutate(Tissue = case_when(Tissue == 'fls' ~ 'Synovium', Tissue == 'lung' ~ 'Lung')), 
        by = c('feature', 'Tissue'), suffix = c('_cluster', '_culture')
    ) %>% 
    identity()
head(plt_df)

In [ ]:
fig.size(4, 8)
p_culture_cor_focus <- plt_df %>% 
    dplyr::mutate(Label = case_when(
        (pvalue_cluster < .01 & (beta) > 1) | (pvalue_culture < .01 & (log2FoldChange) > 1) ~ 'Significant', 
        TRUE ~ 'Not'
    )) %>% 
    dplyr::mutate(Label = factor(Label, c('Significant', 'Not'))) %>% 
    subset(Cluster == '11' & Condition == 'Tcells') %>% 
    ggplot(aes(beta, log2FoldChange)) + 
        geom_point(shape = '.', aes(color = Label)) + 
        geom_hline(yintercept = 0, linetype = 2) + 
        geom_vline(xintercept = 0, linetype = 2) + 
        facet_wrap(~Tissue, scales = 'free') + 
#         guides(color = 'none') + 
#         scale_color_tableau() + 
        scale_color_manual(values = c(muted('red'), 'grey')) + 
        guides(color = guide_legend(override.aes = list(stroke = 1, alpha = 1, shape = 16, size = 4)), alpha = 'none') + 
        labs(x = 'CXCL10+CCL19+ cluster (logFC)', y = 'T cell culture (logFC)') + #, title = 'Comparison of relative expression signatures') + 
        theme(
            legend.title = element_blank(),
            legend.position='bottom'
        ) + 
        geom_point(
            data = . %>% subset(feature %in% c('CXCL10', 'CXCL11', 'CCL19')),             
            size = 2, color = muted('red')
        ) + 
        geom_text_repel(
            data = . %>% subset(feature %in% c('CXCL10', 'CXCL11', 'CCL19')), 
            aes(label = feature),
            max.overlaps = 20,
            box.padding = 1.5,
#             point.padding = 1
#             color = muted('red')
        ) + 
        NULL

p_culture_cor_focus

# Panels

In [ ]:
fig.size(14, 10)
p_final <- (
    p_counts / wrap_elements(full=p_umap_fls | p_umap_lung) / p_scatter_condition / p_culture_cor_focus
) + 

    plot_annotation(tag_levels = 'a')
p_final

In [ ]:
ggsave(
    filename = '/data/srlab2/ik936/Roche/manuscript/figures/SuppFigure7.pdf', 
    plot = p_final,
    units = 'in', device = 'pdf', 
    height = 14, width = 10,
    useDingbats = FALSE
)


ggsave(
    filename = '/data/srlab2/ik936/Roche/manuscript/figures/SuppFigure7.png', 
    plot = p_final,
    units = 'in', device = 'png', 
    height = 14, width = 10
)
